In [1]:
# Load the input trace
# Parallelize the job based on process id
# Combine results

# Loading required libraries
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
import copy
import math
import time
from pathlib import Path
from collections import Counter

# Loading trace
path = r'C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few'
all_files = glob.glob(os.path.join(path, "2016031115-LUN2.csv_expanded_local.csv"))

f = all_files[0]  # Change the file name as required
print("File " + str(f))

cols= ['Timestamp','Response','IOType','LUN','Offset']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols

lba_list = df['Offset'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of IO Accesses :",len(df))
print("Number of unique LBAs : ",len(Counter(lba_list)))

File C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\2016031115-LUN2.csv_expanded_local.csv
Min LBA in the dataset : 135368704
Max LBA in the dataset : 5335731031071
Number of IO Accesses : 5696823
Number of unique LBAs :  2632402


In [3]:
import multiprocessing as mp
from multiprocessing import Process,Queue


def dataprep(lba_list,size_list):
    num_processes = 1
    processes = []
    queue = mp.Queue()
    results = np.ndarray(num_processes)    
    f = all_files[0]
    
    for i in range(num_processes):
        # Generate a sub-list and pass it to the multiprocessing function
        index_start =  int((len(lba_list)/num_processes) * i)
        index_end =  int((len(lba_list)/num_processes)) * (i+ 1) 
        arguments = lba_list,index_start,index_end, i, queue
        p = Process(target = dataprep_multiprocessing, args = arguments)
        p.daemon = True
        processes.append(p)
        processes[i].start()
        
    for i in range(num_processes):
        processes[i].join()
        
    for i in range(num_processes):
        results[i] = queue.get()
        

def dataprep_multiprocessing(new_lba_list,index_start,index_end, i, queue):
    print('Process  Started:', i)
    lba_list = []
    death_time_list = []
    line_counter = 0
    counter = index_start - 1
    print("Starting...")
    while(counter < index_end - 1):
        counter = counter + 1
        line_counter = line_counter + 1
        lba = new_lba_list[counter]
        
        if(line_counter > 5000 and line_counter%5000 == 0):
          print("Process  :",i)
          print("Counter reached  :",line_counter)
          print("Remaining  :",index_end - counter)
          
          new_time_list = df[index_start:counter]['Timestamp'].tolist()
          new_response_list = df[index_start:counter]['Response'].tolist()
          new_lun_list = df[index_start:counter]['LUN'].tolist()
          new_io_list = df[index_start:counter]['IOType'].tolist()

          df_out = pd.DataFrame(columns = ['lba','timestamp','response','io','size','lun','death_time'])
          df_out['lba'] = lba_list
          df_out['death_time'] = death_time_list
          df_out['timestamp'] = new_time_list
          df_out['response'] = new_response_list
          df_out['io'] = new_io_list
          df_out['lun'] = new_lun_list

          df_out.to_csv(f +str(i) +"_deathtime_added.csv",index=False)
          print("File Written.. Still incomplete!!")



        if(new_lba_list[(counter+1):].count(lba) == 0):
            death_time_list.append(-1)
            lba_list.append(lba)
        else:
            next_lba_counter = counter + 1 
            next_lba = new_lba_list[next_lba_counter]
            death_counter = 1
            while(next_lba != lba and next_lba_counter < index_end - 1):
                next_lba_counter = next_lba_counter + 1
                next_lba = new_lba_list[next_lba_counter]
                death_counter = death_counter + 1

            if (next_lba_counter >= len(new_lba_list) - 2):
                death_time_list.append(-1)
                lba_list.append(lba)
            else:
                death_time_list.append(death_counter)
                lba_list.append(lba)
    
      
    new_time_list = df[index_start:index_end]['Timestamp'].tolist()
    new_response_list = df[index_start:index_end]['Response'].tolist()
    new_lun_list = df[index_start:index_end]['LUN'].tolist()
    new_io_list = df[index_start:index_end]['IOType'].tolist()
    new_size_list = df[index_start:index_end]['Size'].tolist()
    df_out = pd.DataFrame(columns = ['lba','timestamp','response','size','io','lun','death_time'])
    df_out['lba'] = lba_list
    df_out['death_time'] = new_size_list
    df_out['size'] = death_time_list
    df_out['timestamp'] = new_time_list
    df_out['response'] = new_response_list
    df_out['io'] = new_io_list
    df_out['lun'] = new_lun_list

    df_out.to_csv(f +str(i) +"_deathtime_added.csv",index=False)
    print("File Written.. Done!!")
    queue.put(0)
    return


In [ ]:
lba_list = df['Offset'].tolist()
size_list = df['Offset'].tolist()
num_processes = 1
k = len(lba_list)%num_processes
lba_list = (lba_list[:-k])
size = (size_list[:-k])
print(len(lba_list))
print(len(lba_list) % num_processes)
dataprep(lba_list,size_list)